In [1]:
import uproot 
import pandas as pd
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tree = uproot.open("givenfile.root:Events")

In [3]:
events =  tree.arrays(library = "ak", how = "zip")

In [4]:
events=events[(events.HLT_Diphoton30_22_R9Id_OR_IsoCaloId_AND_HE_R9Id_Mass90 == 1)|(events.HLT_Diphoton30_22_R9Id_OR_IsoCaloId_AND_HE_R9Id_Mass95 ==1)]

In [5]:
photons = events.Photon

In [6]:
photons.pt

<Array [[107, 18.6, 16.5], ..., [64.6, 21.9]] type='87616 * var * float32'>

In [7]:
def cut(ph):
    x1 = ((ph.isScEtaEB ==1)&(ph.hoe<0.08)&(ph.sieie<0.015)&(ph.r9>0.5)&(ph.r9<0.85)&(ph.pfPhoIso03<4.0)&(ph.trkSumPtHollowConeDR03<6.0))
    x2 = ((ph.isScEtaEB ==1)&(ph.hoe<0.08)&(ph.r9>0.85))
    x3 = ((ph.isScEtaEE ==1)&(ph.hoe<0.08)&(ph.sieie<0.035)&(ph.r9>0.8)&(ph.r9<0.9)&(ph.pfPhoIso03<4.0)&(ph.trkSumPtHollowConeDR03<6.0))
    x4 = ((ph.isScEtaEE ==1)&(ph.hoe<0.08)&(ph.r9>0.9))
    final = ph[((x1|x2)|(x3|x4))]
    return final

In [8]:
photons = cut(photons)

In [9]:
score = photons.pt
score

<Array [[107, 18.6, 16.5], [29.6], ..., [21.9]] type='87616 * var * float32'>

In [10]:
#sorting the data according to pt
photons = photons[ak.argsort(score,ascending = False)]

In [11]:
photons.pt

<Array [[107, 18.6, 16.5], [29.6], ..., [21.9]] type='87616 * var * float32'>

In [12]:
photons = photons[(ak.where(ak.count(photons.pt,axis=1)>1))]
photons.pt

<Array [[107, 18.6, 16.5], ..., [63.3, 33.6]] type='36305 * var * float32'>

In [13]:
diphoton = ak.combinations(photons,2,fields = ["leading","subleading"])
diphoton.leading.pt

<Array [[107, 107, 18.6], [122], ..., [63.3]] type='36305 * var * float32'>

In [14]:
diphoton.subleading.pt

<Array [[18.6, 16.5, 16.5], [24.6], ..., [33.6]] type='36305 * var * float32'>

In [15]:
#mass of the pair
diphoton["pair_mass"] = np.sqrt((2*diphoton.leading.pt*diphoton.subleading.pt)*(np.cosh(diphoton.leading.eta - diphoton.subleading.eta) - np.cos(diphoton.leading.phi - diphoton.subleading.phi)))
diphoton.pair_mass

<Array [[3.8, 1.92, 1.15], [146], ..., [115]] type='36305 * var * float32'>

In [16]:
def select(diphoton):
    con = ((np.sqrt(diphoton.leading.pt*diphoton.subleading.pt)/diphoton.pair_mass > 1/3)&(diphoton.subleading.pt/diphoton.pair_mass > 1/4))
    new = diphoton[con]
    return new
diphoto = select(diphoton)
diphoto.subleading.pt

<Array [[18.6, 16.5, 16.5], [], ..., [32], [33.6]] type='36305 * var * float32'>

In [17]:
diphoton = diphoton[ak.argmax(diphoton.leading.pt + diphoton.subleading.pt,axis=1,keepdims = True)]
diphoton.leading.pt

<Array [[107], [122], ..., [35.9], [63.3]] type='36305 * var * ?float32'>

In [18]:
#photon1_df = ak.to_dataframe(diphoton[:,0])
#photon1_df.to_csv('photon1.csv')

In [19]:
lead = diphoton.leading
sublead = diphoton.subleading
mass = diphoton.pair_mass
sublead.pt

<Array [[18.6], [24.6], ..., [32], [33.6]] type='36305 * var * ?float32'>

In [20]:
# ... (Your code to define 'lead' from 'diphoton.leading')
fields_to_keep = lead.fields
# 1. Flatten and Simplify
flattened_lead = ak.flatten(lead)  # Flatten nested structure if present

# 2. Extract Relevant Fields  # Add other fields as needed
lead_for_df = ak.zip({field: flattened_lead[field] for field in fields_to_keep})

# 3. Convert to DataFrame
df1 = ak.to_dataframe(lead_for_df)
df1 = df1.add_prefix('leadpho_')

In [21]:
# ... (Your code to define 'lead' from 'diphoton.leading')
fields_to_keep = sublead.fields
# 1. Flatten and Simplify
flattened_sublead = ak.flatten(sublead)  # Flatten nested structure if present

# 2. Extract Relevant Fields  # Add other fields as needed
sublead_for_df = ak.zip({field: flattened_sublead[field] for field in fields_to_keep})

# 3. Convert to DataFrame
df2 = ak.to_dataframe(sublead_for_df)
df2 = df2.add_prefix('subleadpho_')
df2['subleadpho_pt']

entry
0        18.641722
1        24.609812
2        32.259377
3        23.982353
4        51.655556
           ...    
36300    37.249718
36301    31.510630
36302    37.343422
36303    32.034779
36304    33.616512
Name: subleadpho_pt, Length: 36305, dtype: float32

In [22]:
mass_for_df=ak.flatten(mass)
mass_df = ak.to_dataframe(mass_for_df)
mass_df

,values
entry,
0,3.797012
1,146.366074
2,336.995605
3,107.637794
4,187.100769
...,...
36300,113.451836
36301,92.719330
36302,127.054420


In [23]:
mass_df.rename(columns = {'values':'pair_mass'}, inplace = True)
mass_df

,pair_mass
entry,
0,3.797012
1,146.366074
2,336.995605
3,107.637794
4,187.100769
...,...
36300,113.451836
36301,92.719330
36302,127.054420


In [24]:
final = pd.concat([df1, df2,mass_df], axis=1, join = 'outer')
final.to_csv('photon_pair_data.csv')